# Earthquakes near Istanbul, Turkey in 2019

## Dataset from [Kaggle](https://www.kaggle.com/mielek/earthquakes-around-istanbul-in-2019)

### Included in the dataset:
* Event ID
* Time
* Latitude
* Longitude
* Depth/km
* Magnitude Type
* Magnitude
* Event Location Name

Sources:
[MTU](https://www.mtu.edu/geo/community/seismology/learn/earthquake-measure/magnitude/)
[Medium](https://medium.com/mlearning-ai/earthquakes-data-visualization-5da0d2abeea1)

## Imports

In [121]:
import pandas as pd

import seaborn as sns
sns.set_style("whitegrid")

import matplotlib.pyplot as plt
%matplotlib inline

import folium 
from folium.plugins import HeatMap

import plotly.graph_objects as go
import plotly.express as px

## Data prep

In [122]:
df = pd.read_csv("deprem.csv")
df.head()

,#EventID,Time,Latitude,Longitude,Depth/km,MagType,Magnitude,EventLocationName
0,20200204_0000090,2020-02-04T10:17:09.4Z,38.98,27.86,6,ml,3.4,WESTERN TURKEY
1,20200204_0000082,2020-02-04T08:43:16.0Z,38.98,27.87,10,ml,4.1,WESTERN TURKEY
2,20200204_0000067,2020-02-04T06:13:29.2Z,38.42,25.51,15,ml,3.3,AEGEAN SEA
3,20200204_0000046,2020-02-04T03:42:21.5Z,38.40,25.53,17,ml,3.3,AEGEAN SEA
4,20200204_0000034,2020-02-04T03:14:45.4Z,39.08,27.83,9,ml,3.3,WESTERN TURKEY


### Set location

In [123]:
coord = df[[" Latitude "," Longitude "]]

In [124]:
coord.head()

,Latitude,Longitude
0,38.98,27.86
1,38.98,27.87
2,38.42,25.51
3,38.40,25.53
4,39.08,27.83


In [125]:
coord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 2 columns):
 Latitude      349 non-null float64
 Longitude     349 non-null float64
dtypes: float64(2)
memory usage: 5.5 KB


In [126]:
coord.describe()

,Latitude,Longitude
count,349.000000,349.000000
mean,39.467937,27.695043
std,0.867710,0.959359
min,38.000000,25.330000
25%,39.010000,27.630000
50%,39.070000,27.840000
75%,39.660000,27.900000
max,43.420000,31.510000


### Drop NaN

In [127]:
coord = coord.dropna()

In [128]:
coord.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349 entries, 0 to 348
Data columns (total 2 columns):
 Latitude      349 non-null float64
 Longitude     349 non-null float64
dtypes: float64(2)
memory usage: 8.2 KB


## Create HeatMap feat. folium

### Avg. latitude and longitude to set the location

In [129]:
lat = coord[" Latitude "].mean()
long = coord[" Longitude "].mean()

In [130]:
hmap = folium.Map(location=[lat,long],zoom_start=6)
hmap.add_child(HeatMap(coord, radius = 15))

hmap

## Earthquake Magnitudes

In [131]:
df[" Magnitude "].value_counts()

3.1    66
3.0    57
3.2    50
3.3    33
3.4    31
3.5    21
3.7    17
3.6    16
4.0    10
3.9     9
3.8     8
4.1     6
4.2     4
4.4     4
4.6     3
4.7     3
4.3     3
4.9     3
4.5     3
5.7     1
5.6     1
Name:  Magnitude , dtype: int64

* <2.5      : Recorded by seismograph, usually not felt. Millions every year.
* 2.5 - 5.4	: Often felt, but causes minor damage. ~500,000 every year.
* 5.5 - 6.0	: Slight damage to buildings and structures. ~350 every year.
* 6.1 - 6.9	: May cause damage in densely populated areas. ~100 every year.	
* 7.0 - 7.9	: Major earthquake. Serious damage.	~10-15 every year.
* 8.0 +     : Great earthquake. Can destroy communities near the epicenter.	1 every 1-2 years.

Source: [MTU](https://www.mtu.edu/geo/community/seismology/learn/earthquake-measure/magnitude/)

## Where do earthquakes tend to occur?

In [132]:
df["region"] = df[" EventLocationName"].apply(lambda x: str(x[0:2]))
df["region"].value_counts()

WE    284
NE     33
AE     26
BL      3
BU      3
Name: region, dtype: int64

* High density of earthquakes around western Turkey.

## Worldwide Earthquake visualization

In [133]:
def get_date(time):
    return str(time).split(" ")[0]

def get_day(time):
    date = get_date(time)
    return date + " - " + str(time.weekday())

def get_hour(time):
    t = str(time).split(' ')
    return t[0] + " - " + t[1].split(":")[0]


def get_subarea(x):
    return x[0]

def get_area(x):
    return x[-1]

In [134]:
def get_quakes(timefr="daily", bbox='Worldwide', mag_thresh=2):
    url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/{}.csv"
    
    if (timefr == "weekly"):
        link = url.format("all_week")
    elif (timefr == "monthly"):
        link = url.format("all_month")
    else:
        link = url.format("all_day")
    
    quake_df = pd.read_csv(filepath_or_buffer = link)
    quake_df = quake_df[["time", "latitude", "longitude", "mag", "place"]]
    
    place_list = quake_df["place"].str.split(", ")
    quake_df["sub_area"] = place_list.apply(func = get_subarea)
    quake_df["area"] = place_list.apply(func = get_area)
    quake_df = quake_df.drop(columns=["place"], axis=1)
    
    if isinstance(mag_thresh, int) and (mag_thresh > 0):
        quake_df = quake_df[quake_df["mag"] >= mag_thresh]
    else:
        quake_df = quake_df[quake_df["mag"] > 0]
    
    if bbox in quake_df["area"].to_list():
        quake_df = quake_df[quake_df["area"] == bbox]
        max_mag = quake_df["mag"].max()
        center_lat = quake_df[quake_df["mag"] == max_mag]["latitude"].values[0]
        center_lon = quake_df[quake_df["mag"] == max_mag]["longitude"].values[0]
    else:
        center_lat, center_lon = [38, 35]
    
    quake_df["time"] = pd.to_datetime(quake_df["time"])
    
    if (timefr == "weekly"):
        new_col = "weekday"
        quake_df[new_col] = quake_df["time"].apply(get_day)
    elif (timefr == "monthly"):
        new_col = "date"
        quake_df[new_col] = quake_df["time"].apply(get_date)
    else:
        new_col = "hours"
        quake_df[new_col] = quake_df["time"].apply(get_hour)
    
    quake_df = quake_df.sort_values(by="time")
    
    return quake_df, center_lat, center_lon

In [135]:
def visualize_quakes(timefr="daily", bbox="Worldwide", mag_thresh=2):
    quake_df, clat, clon = get_quakes(timefr = timefr, bbox = bbox, mag_thresh = mag_thresh)
    
    if (timefr == "monthly"):
        af = "date"
    elif (timefr == "weekly"):
        af = "weekday"
    else:
        af = "hours"
    
    zoom = 3 if bbox != "Worldwide" else 1
    
    fig = px.scatter_mapbox(
        data_frame = quake_df,
        lat = "latitude",
        lon = "longitude",
        center = dict(lat=clat, lon=clon),
        size = "mag",
        color = "mag",
        hover_name = "sub_area",
        zoom = zoom,
        mapbox_style = "stamen-terrain",

        animation_frame=af,
        title ="{} Earthquakes - {}".format(timefr.title(), bbox)
    )
    
    fig.update_layout(
        margin = dict(l=0, r=0, t=40, b=0)
    )
    
    fig.show()
    return None

In [136]:
visualize_quakes('monthly')